# Description

A __Hopfield network__ consists of $N$ fully connected neurons. The model states __evolve__ in time with time steps $\Delta t$ (here it's assumed that $\Delta t = 1$).


The __state__ of the netwok at time $t$ is denoted by a vector

$$ \mathbf{S}(t) = \{ S_1 (t), \ S_2(t), \ \dots, S_N(t) \} $$

where each state has __two possible states__ $S_i(t) \in \{-1, 1\}, \forall i \in \{1, \dots, N \}$. 


<br>


Neurons interact with each other with weights $w_{i,j}$, where $i, j \in \{1, \dots, N \}$. These weights can be expressed as matrix $\mathbf{W} \in \mathbb{R}^{N \times N}$.


The state $s_i$ of each neuron is __updated with rule__

$$
S_i(t + 1) = \text{sgn}[h_i(t)] =
\begin{cases}
1 & \text{if } h_i(t) > 0 \\
S_i(t) & \text{if } h_i(t) = 0 \\
-1 & \text{if } h_i(t) < 0 \\
\end{cases}
$$

with 

$$ h_i(t) = \sum_{j=1}^N w_{ij} S_j(t) - \theta $$

with $\theta = 0$

In the standard model, __asynchronous updates__ are used to ensure the network reaches a stable configuration $\mathbf{S}(t + 1) = \mathbf{S}(t)$.




# Patterns in Hopfield Network

A Hopfield Network acts as __associative memory__ where stored __patterns__ are represented as a __set of neuron configurations__ $\mathbf{p}^{(\mu)} \in \{-1, 1\}^N, \mu \in \{1, \dots, M \}$. The network dynamics are designed such that these patterns become __attractors__ in the state space.


Gievn an initial setup $\mathbf{S}(t = 0) = \mathbf{p}$ (a distorted version of $\mathbf{p}^{(\mu)}$), the system evolves over time $t$ such that:

$$ \lim_{t \rightarrow \infty} \mathbf{S}(t) = \mathbf{p}^{(\mu)} $$
 

In [1]:
import random
import numpy as np

random.seed(1)

N = 8
M = 2

T = np.zeros((M, N))

for i in range(M):
    for j in range(N):
        T[i][j] = random.choice([-1, 1])

print(T)

[[-1. -1.  1. -1.  1.  1.  1.  1.]
 [-1. -1.  1. -1.  1.  1. -1.  1.]]


To store $M$ patterns in the network, the weights are determined using a __Hebbian learning rule__. The weight $w_{ij}$ represents the association between neurons $i$ and $j$ across all stored patterns

$$ w_{ij} = c\sum_{\mu=1}^M p_i^{(\mu)} p_j^{(\mu)}, \ \ \ \ \ c \in \mathbb{R}^+, \ \forall i, j \in {1, \dots, N} $$


In matrix form, this is represented as the sum of the __outer products__ of all pattern vectors

$$ \mathbf{W} = c \sum_{\mu=1}^M \mathbf{p}^{(\mu)}[\mathbf{p}^{(\mu)}]^T $$


standard choice of the constant $c$ is $c=1/N$.


To ensure the network does not coverge to trivial solution and to maintain the properties of a Energy function, we enforce:
1. __Symmetry__: $w_{ij} = w_{ji}$ (property of $\mathbf{x}\mathbf{x}^T$)
2. __No self connection__: $w_{ii} = 0, \ \forall i$ 

The modified weight matrix satisfying these constraints is: 

$$ \mathbf{W} = \frac{1}{N} \sum_{\mu=1}^M \big( \mathbf{p}^{(\mu)}[\mathbf{p}^{(\mu)}]^T - \mathbf{I} \big) $$


# Pattern Retrieval

The measure of similarity between current state $\mathbf{S}(t)$ and pattern $\mathbf{p}^{(\mu)}$ is calculated using __overlap parameter__ (magnetization):

$$ m^{(\mu)} (t) =  \frac{1}{N} \sum_{i = 1}^N p_i^{(\mu)} S_i(t) =  \frac{1}{N} [\mathbf{p}^{(\mu)}]^T \mathbf{S}(t)  $$


<br>

We can analyze the local field $h_i(t)$ in terms of these overlaps. Substituting the Hebbian weights:

$$ h_i(t) = \sum_{j=1}^N w_{ij} S_j(t) = \sum_{j=1}^N \left( c\sum_{\nu=1}^M p_i^{(\nu)} p_j^{(\nu)} \right) S_j(t) $$

Rearranging the summation order:

$$h_i(t) = cN\sum_{\nu=1}^M p_i^{(\nu)} \underbrace{\left( \frac{1}{N} \sum_{j=1}^N p_j^{(\nu)} S_j(t) \right)}_{m^{(\nu)}(t)}$$


$$ h_i(t) = cN \sum_{\nu = 1}^M p_i^{(\nu)} m^{(\nu)}(t)$$

Note that the above calculation naturally implies $c = 1/N$ to ensure the local field remains independent of the network size $N$.
